<a href="https://colab.research.google.com/github/Graciela-ia/Algoritmos/blob/main/Seminario_Algoritmos_Graciela.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Graciela Prada Ruiz  <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---2019/tree/master/SEMINARIO<br>
Problema:

>3. Combinar cifras y operaciones

Descripción del problema:

• El problema consiste en analizar el siguiente problema y diseñar un algoritmo que lo resuelva.<br>
• Disponemos de las 9 cifras del 1 al 9 (excluimos el cero) y de los 4 signos básicos de las
operaciones fundamentales: suma(+), resta(-), multiplicación(*) y división(/)<br>
• Debemos combinarlos alternativamente sin repetir ninguno de ellos para obtener una
cantidad dada. Un ejemplo sería para obtener el 4:<br>
     <pre>4+2-6/3x1 = 4</pre>
• Debe analizarse el problema para encontrar todos los valores enteros posibles planteando las
siguientes cuestiones:
- ¿Qué valor máximo y mínimo se pueden obtener según las condiciones del problema?
- ¿Es posible encontrar todos los valores enteros posibles entre dicho mínimo y máximo?

(*) La respuesta es obligatoria





                                        

In [13]:
# Configuración inicial e imports necesarios
import random
import json
from math import perm, factorial
from itertools import permutations

# Configuración para mejor visualización
import warnings
warnings.filterwarnings('ignore')

print("Configuración inicial completada")
print("Librerías importadas correctamente")


Configuración inicial completada
Librerías importadas correctamente


## Resolución:
### Planteamiento:
Para resolver este problema debemos entender primero lo que nos piden: El objetivo es encontrar todas las combinaciones posibles de cifras del 1 al 9 y los operadores +, -, *, / (sin repetir ninguno), que den como resultado un número entero.
Dado:<br>
- Cifras: {1, 2, 3, 4, 5, 6, 7, 8, 9}<br>
- Operadores: {'+', '-', '*', '/'}<br>
Reglas:<br>
- Usar cada cifra exactamente una vez.<br>
- Usar cada operador exactamente una vez.<br>
- Alternar cifras y operadores: por ejemplo: c + c - c / c * c ...<br>
- No usar paréntesis.<br>
- Evaluar la expresión sin prioridad de operadores (como se lee de izquierda a derecha).<br>
- Buscar todos los valores enteros posibles que se pueden obtener con estas reglas.<br>


Para saber el número de cifras necesarias, al tener que alternar con los operadores, serán necesarias un total de operadores más 1, para que queden alternados sin repetir:

In [14]:
# Número de operadores disponibles
num_operadores = len(['+', '-', '*', '/'])

# Para alternar correctamente: cifra op cifra op ... cifra
# Se necesita que:
#   - cantidad de cifras = cantidad de operadores + 1

num_cifras_necesarias = num_operadores + 1

print(f"Para {num_operadores} operadores, se necesitan {num_cifras_necesarias} cifras.")

Para 4 operadores, se necesitan 5 cifras.


(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




Respuesta

**Si no imponemos restricciones:**

Se pueden usar repetidamente cifras y operadores.
La secuencia es: c op c op c op c op c → 5 cifras y 4 operadores
Entonces:

Cada cifra tiene 9 opciones (del 1 al 9)
Cada operador tiene 4 opciones
Total de combinaciones:
9^5 × 4^4 = 59049×256=15,116,544

In [15]:
# Posibilidades sin restricciones
total_sin_restricciones = (9**5) * (4**4)
print(f"Posibilidades sin restricciones: {total_sin_restricciones}")

Posibilidades sin restricciones: 15116544


**Si tenemos en cuenta las restricciones:**


Usar 5 cifras distintas de {1..9}
Usar los 4 operadores , todos distintos (permutaciones)

Cálculo:
Número de permutaciones de 5 cifras de 9:
P(9,5)= (9−5)! / 9! = 15120

Número de permutaciones de los 4 operadores:
4!=24

Total de combinaciones:
15120×24=362880

In [16]:
#from math import perm, factorial

# Con restricciones
pos_con_restricciones = perm(9,5) * factorial(4)
print(f"Posibilidades con restricciones: {pos_con_restricciones}")

Posibilidades con restricciones: 362880


Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, argumentalo)


Respuesta

La estructura de datos elegida es:
- Tuplas de 5 números seleccionados de {1..9}, sin repetición.
- Tuplas de 4 operadores , permutaciones de {'+', '-', '*', '/'}
- Resultados guardados en un conjunto (set)

Utilizaremos:
- itertools.permutations: porque permite generar todas las combinaciones sin repetición.
- set: porque evita duplicados y facilita el conteo de resultados únicos.
- Tuplas: ya que son inmutables, ideales para claves o almacenamiento seguro.

In [17]:
# Definición del conjunto de cifras y operadores
cifras = list(range(1, 10))   # [1, 2, ..., 9]
operadores = ['+', '-', '*', '/']

# Estructura de datos elegida: permutaciones de 5 cifras y permutaciones de los 4 operadores
# Esto asegura que no haya repetición de cifras ni operadores

# Ejemplo de una solución válida:
una_cifra_ejemplo = (4, 2, 6, 3, 1)        # Tupla de 5 cifras distintas
un_operador_ejemplo = ('+', '-', '/', '*') # Tupla de 4 operadores en un orden dado

# Representación de una solución completa:
# (c1, c2, c3, c4, c5), (op1, op2, op3, op4)
print(f"Ejemplo de una solución válida: {una_cifra_ejemplo}, {un_operador_ejemplo}")



Ejemplo de una solución válida: (4, 2, 6, 3, 1), ('+', '-', '/', '*')


Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta

Función objetivo: Dada una expresión formada por 5 cifras y 4 operadores, evaluar su resultado numérico usando evaluación izquierda a derecha.

In [18]:
def evaluar_expresion(cifr, ops):
    """
    Evalua la expresión formada por las cifras y operadores,
    siguiendo el orden de izquierda a derecha, sin prioridad.
    """
    resultado = cifr[0]
    for i in range(4):
        op = ops[i]
        b = cifr[i + 1]
        if op == '+':
            resultado += b
        elif op == '-':
            resultado -= b
        elif op == '*':
            resultado *= b
        elif op == '/':
            if b == 0:
                return None  # Evitar división por cero
            resultado /= b
    return resultado

# Ejemplo de uso de la función con los ejemplos anteriores
valor_resultante = evaluar_expresion(una_cifra_ejemplo, un_operador_ejemplo)
print(f"Evaluando {una_cifra_ejemplo} con {un_operador_ejemplo}: Resultado = {valor_resultante}")

Evaluando (4, 2, 6, 3, 1) con ('+', '-', '/', '*'): Resultado = 0.0


No es estrictamente un problema de optimización en sentido tradicional (no se busca maximizar ni minimizar una función), sino un problema de búsqueda exhaustiva de valores enteros alcanzables.

Sin embargo, si queremos verlo como problema de optimización, podríamos definir una función objetivo que mida cómo de cerca estamos de un valor dado.
En ese caso, sería un problema de minimización de error entre el resultado obtenido y el deseado.

In [19]:
# Valor objetivo que queremos alcanzar
valor_objetivo = 100

# Función objetivo: calcula el error absoluto entre el resultado y el valor objetivo
def funcion_objetivo(cifras_comb, ops_comb, objetivo):
    resultado = evaluar_expresion(cifras_comb, ops_comb)
    if resultado is None:
        return float('inf')  # Penalizamos la división por cero
    return abs(resultado - objetivo)


# Búsqueda exhaustiva para minimizar el error
mejor_error = float('inf')
mejor_solucion = None

for nums in permutations(cifras, 5):
    for ops in permutations(operadores):
        error = funcion_objetivo(nums, ops, valor_objetivo)
        if error < mejor_error:
            mejor_error = error
            mejor_solucion = (nums, ops, evaluar_expresion(nums, ops))

# Mostrar la mejor solución encontrada
print("Mejor solución encontrada:")
print(f"Cifras: {mejor_solucion[0]}")
print(f"Operadores: {mejor_solucion[1]}")
print(f"Resultado obtenido: {mejor_solucion[2]}")
print(f"Valor objetivo: {valor_objetivo}")
print(f"Error absoluto: {mejor_error}")

Mejor solución encontrada:
Cifras: (5, 1, 7, 9, 8)
Operadores: ('/', '+', '*', '-')
Resultado obtenido: 100.0
Valor objetivo: 100
Error absoluto: 0.0


Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

In [20]:
#Algoritmo por fuerza bruta

results = set()

for nums in permutations(cifras, 5):  # 15120
    for ops in permutations(operadores):  # 24
        value = nums[0]
        try:
            for i in range(4):
                b = nums[i+1]
                op = ops[i]
                if op == '+': value += b
                elif op == '-': value -= b
                elif op == '*': value *= b
                elif op == '/': value /= b
            if value == int(value):
                results.add(int(value))
        except ZeroDivisionError:
            continue

Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

Número total de iteraciones:
P(9,5)×4!=15120×24=362880
Cada iteración realiza 4 operaciones aritméticas → O(1) por iteración
Complejidad temporal:

O(P(9,5)⋅4!)=O(362880)
Espacio:

Almacenamiento de resultados: O(k), donde k es el número de enteros diferentes obtenidos (en práctica < 1000)

In [21]:
# Número de cifras disponibles (1 al 9)
total_cifras = 9

# Seleccionamos 5 cifras sin repetición
num_cifras_seleccionadas = 5

# Número de operadores disponibles (+, -, *, /)
num_operadores = 4

# Cálculo del número total de combinaciones posibles
permutaciones_cifras = perm(total_cifras, num_cifras_seleccionadas)  # P(9,5)
permutaciones_operadores = factorial(num_operadores)                # 4!

# Total de expresiones a evaluar
total_combinaciones = permutaciones_cifras * permutaciones_operadores

# Complejidad asintótica en notación O grande
print("Cálculo de la complejidad del algoritmo por fuerza bruta:")
print(f"Número de permutaciones de 5 cifras de {total_cifras}: P({total_cifras}, {num_cifras_seleccionadas}) = {permutaciones_cifras}")
print(f"Número de permutaciones de {num_operadores} operadores: {num_operadores}! = {permutaciones_operadores}")
print(f"Total de combinaciones a evaluar: {total_combinaciones}")

# Mostramos la complejidad en notación Big O
print("\nComplejidad del algoritmo por fuerza bruta:")
print("O(P(n, k) x m!) → donde:")
print(f"  n = {total_cifras} (total de cifras)")
print(f"  k = {num_cifras_seleccionadas} (cifras seleccionadas)")
print(f"  m = {num_operadores} (operadores)")

# Justificación
print("\nJustificación:")
print("El algoritmo prueba todas las combinaciones posibles de 5 cifras distintas")
print("del conjunto {1..9} y todas las permutaciones de los 4 operadores básicos.")
print("Cada una de estas combinaciones requiere una evaluación aritmética de 4 pasos.")
print("Por lo tanto, la complejidad es proporcional al número total de combinaciones.")

Cálculo de la complejidad del algoritmo por fuerza bruta:
Número de permutaciones de 5 cifras de 9: P(9, 5) = 15120
Número de permutaciones de 4 operadores: 4! = 24
Total de combinaciones a evaluar: 362880

Complejidad del algoritmo por fuerza bruta:
O(P(n, k) x m!) → donde:
  n = 9 (total de cifras)
  k = 5 (cifras seleccionadas)
  m = 4 (operadores)

Justificación:
El algoritmo prueba todas las combinaciones posibles de 5 cifras distintas
del conjunto {1..9} y todas las permutaciones de los 4 operadores básicos.
Cada una de estas combinaciones requiere una evaluación aritmética de 4 pasos.
Por lo tanto, la complejidad es proporcional al número total de combinaciones.


(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

Para mejorar la complejidad del algoritmo por fuerza bruta podemos reducir el número de evaluaciones necesarias, aplicando ténica de poda temprana, ya que si detectamos que el valor intermedio ya es demasiado grande o claramente no entero, dejamos de evaluar esa expresión. Ademas, saltamos las combinaciones donde haya división por cero.

In [22]:
# Función que evalúa una expresión con poda temprana
def evaluar_con_poda(cifras_comb, ops_comb, limite=100000):
    try:
        valor = cifras_comb[0]
        for i in range(4):
            op = ops_comb[i]
            b = cifras_comb[i + 1]
            if op == '+':
                valor += b
            elif op == '-':
                valor -= b
            elif op == '*':
                valor *= b
            elif op == '/':
                valor /= b
            # Poda temprana si el valor es demasiado grande
            if abs(valor) > limite:
                return None, "poda"
        return valor, "entero" if valor == int(valor) else "no_entero"
    except ZeroDivisionError:
        return None, "division_por_cero"

(*)Calcula la complejidad del algoritmo

Respuesta

In [23]:
# Parámetros del problema
total_cifras = 9
num_cifras_seleccionadas = 5
num_operadores = 4

# Cálculos combinatorios
permutaciones_cifras = perm(total_cifras, num_cifras_seleccionadas)
permutaciones_operadores = factorial(num_operadores)
total_combinaciones = permutaciones_cifras * permutaciones_operadores

# Estimación de la mejora debido a la poda temprana
# Supongamos un porcentaje estimado de poda (ajustable)
porcentaje_poda_estimado = 0.25  # 25% de evaluaciones evitadas

# Número efectivo de evaluaciones completas
evaluaciones_efectivas = total_combinaciones * (1 - porcentaje_poda_estimado)

print("Complejidad del algoritmo mejorado")
print("─" * 34)
print(f"Número total de combinaciones posibles: {total_combinaciones}")
print(f"Porcentaje estimado de poda temprana: {porcentaje_poda_estimado * 100}%")
print(f"Número efectivo de evaluaciones completas: {int(evaluaciones_efectivas)}")
print("\nComplejidad:")
print("En notación O grande:")
print("  Peor caso: O(P(n, k) x m!) → O(362880)")
print("  Caso promedio (con poda): ~75% del peor caso")
print("  Donde n=9, k=5, m=4")

print("\nJustificación:")
print("Aunque el número total de combinaciones sigue siendo el mismo que en la fuerza bruta,")
print("la poda temprana permite evitar evaluar completamente ciertas expresiones.")
print("Esto reduce el tiempo real de ejecución, aunque la complejidad teórica permanezca invariable.")

Complejidad del algoritmo mejorado
──────────────────────────────────
Número total de combinaciones posibles: 362880
Porcentaje estimado de poda temprana: 25.0%
Número efectivo de evaluaciones completas: 272160

Complejidad:
En notación O grande:
  Peor caso: O(P(n, k) x m!) → O(362880)
  Caso promedio (con poda): ~75% del peor caso
  Donde n=9, k=5, m=4

Justificación:
Aunque el número total de combinaciones sigue siendo el mismo que en la fuerza bruta,
la poda temprana permite evitar evaluar completamente ciertas expresiones.
Esto reduce el tiempo real de ejecución, aunque la complejidad teórica permanezca invariable.


Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

In [24]:

# Generar un juego de datos aleatorio
def generar_juego_aleatorio():
    nums = tuple(random.sample(cifras, 5))  # 5 cifras distintas
    ops = tuple(random.sample(operadores, 4))  # 4 operadores en orden aleatorio
    return nums, ops

# Generar y almacenar múltiples juegos de datos
def generar_conjunto_juegos(n=20):
    """Genera n juegos aleatorios y los almacena en una lista"""
    juegos = []
    for i in range(n):
        nums, ops = generar_juego_aleatorio()
        juegos.append((nums, ops))
        print(f"Juego {i+1}:")
        print(f"  Cifras: {nums}")
        print(f"  Operadores: {ops}")
    return juegos

# Generar y almacenar 10 juegos aleatorios
print("Generando conjunto de juegos de datos:")
print("=" * 40)
juegos_almacenados = generar_conjunto_juegos(20)

Generando conjunto de juegos de datos:
Juego 1:
  Cifras: (7, 3, 5, 2, 8)
  Operadores: ('/', '+', '-', '*')
Juego 2:
  Cifras: (5, 8, 6, 9, 1)
  Operadores: ('-', '*', '/', '+')
Juego 3:
  Cifras: (8, 2, 4, 7, 1)
  Operadores: ('/', '+', '-', '*')
Juego 4:
  Cifras: (3, 5, 9, 7, 2)
  Operadores: ('+', '/', '*', '-')
Juego 5:
  Cifras: (7, 9, 3, 5, 6)
  Operadores: ('+', '*', '-', '/')
Juego 6:
  Cifras: (8, 4, 1, 7, 2)
  Operadores: ('*', '-', '/', '+')
Juego 7:
  Cifras: (2, 8, 7, 6, 3)
  Operadores: ('/', '-', '*', '+')
Juego 8:
  Cifras: (8, 4, 1, 6, 5)
  Operadores: ('-', '*', '/', '+')
Juego 9:
  Cifras: (5, 4, 1, 2, 7)
  Operadores: ('+', '-', '*', '/')
Juego 10:
  Cifras: (3, 6, 4, 9, 2)
  Operadores: ('+', '/', '*', '-')
Juego 11:
  Cifras: (5, 9, 2, 8, 3)
  Operadores: ('-', '*', '/', '+')
Juego 12:
  Cifras: (1, 3, 7, 9, 4)
  Operadores: ('/', '*', '+', '-')
Juego 13:
  Cifras: (1, 3, 6, 8, 9)
  Operadores: ('/', '+', '-', '*')
Juego 14:
  Cifras: (5, 1, 8, 3, 2)
  Operadore

Aplica el algoritmo al juego de datos generado

Respuesta

In [26]:

# Aplicar algoritmo a juegos previamente almacenados
def aplicar_algoritmo_a_juegos_almacenados(juegos_datos):
    """Aplica el algoritmo de evaluación a un conjunto de juegos previamente generados"""
    print(f"Aplicando algoritmo a {len(juegos_datos)} juegos almacenados:\n")
    resultados = []

    for i, (nums, ops) in enumerate(juegos_datos):
        resultado, tipo = evaluar_con_poda(nums, ops)

        # Almacenar el resultado completo
        resultado_completo = {
            'juego_id': i + 1,
            'cifras': nums,
            'operadores': ops,
            'resultado': resultado,
            'tipo': tipo,
            'es_entero': resultado == int(resultado) if tipo not in ["poda", "division_por_cero"] else False
        }
        resultados.append(resultado_completo)

        print(f"Juego {i+1}:")
        print(f"  Cifras: {nums}")
        print(f"  Operadores: {ops}")

        if tipo == "poda":
            print("  → Valor excedido. Evaluación abortada (poda temprana).")
        elif tipo == "division_por_cero":
            print("  → División por cero. Expresión inválida.")
        else:
            print(f"  Resultado: {resultado}")
            if resultado == int(resultado):
                print("  ✓ SÍ - Es un número entero.")
            else:
                print("  ✗ NO - No es un número entero.")
        print("-" * 40)

    return resultados

# Aplicar el algoritmo a los juegos previamente almacenados
print("\nAplicando algoritmo a los juegos almacenados:")
print("=" * 50)
resultados_obtenidos = aplicar_algoritmo_a_juegos_almacenados(juegos_almacenados)


Aplicando algoritmo a los juegos almacenados:
Aplicando algoritmo a 20 juegos almacenados:

Juego 1:
  Cifras: (7, 3, 5, 2, 8)
  Operadores: ('/', '+', '-', '*')
  Resultado: 42.66666666666667
  ✗ NO - No es un número entero.
----------------------------------------
Juego 2:
  Cifras: (5, 8, 6, 9, 1)
  Operadores: ('-', '*', '/', '+')
  Resultado: -1.0
  ✓ SÍ - Es un número entero.
----------------------------------------
Juego 3:
  Cifras: (8, 2, 4, 7, 1)
  Operadores: ('/', '+', '-', '*')
  Resultado: 1.0
  ✓ SÍ - Es un número entero.
----------------------------------------
Juego 4:
  Cifras: (3, 5, 9, 7, 2)
  Operadores: ('+', '/', '*', '-')
  Resultado: 4.222222222222221
  ✗ NO - No es un número entero.
----------------------------------------
Juego 5:
  Cifras: (7, 9, 3, 5, 6)
  Operadores: ('+', '*', '-', '/')
  Resultado: 7.166666666666667
  ✗ NO - No es un número entero.
----------------------------------------
Juego 6:
  Cifras: (8, 4, 1, 7, 2)
  Operadores: ('*', '-', '/', 

In [27]:
# Análisis de los resultados obtenidos
def analizar_resultados(resultados):
    """Analiza los resultados obtenidos de los juegos evaluados"""
    total_juegos = len(resultados)
    enteros_obtenidos = [r for r in resultados if r['es_entero']]
    divisiones_por_cero = [r for r in resultados if r['tipo'] == 'division_por_cero']
    podas = [r for r in resultados if r['tipo'] == 'poda']

    print("\n" + "=" * 50)
    print("ANÁLISIS DE RESULTADOS")
    print("=" * 50)
    print(f"Total de juegos evaluados: {total_juegos}")
    print(f"Resultados enteros obtenidos: {len(enteros_obtenidos)}")
    print(f"Divisiones por cero: {len(divisiones_por_cero)}")
    print(f"Evaluaciones con poda temprana: {len(podas)}")

    if enteros_obtenidos:
        print("\nVALORES ENTEROS ENCONTRADOS:")
        for resultado in enteros_obtenidos:
            print(f"  Juego {resultado['juego_id']}: {resultado['resultado']}")

        valores_unicos = set([int(r['resultado']) for r in enteros_obtenidos])
        print(f"\nValore únicos enteros: {sorted(valores_unicos)}")
        print(f"Cantidad de valores únicos: {len(valores_unicos)}")

    return {
        'total': total_juegos,
        'enteros': len(enteros_obtenidos),
        'divisiones_cero': len(divisiones_por_cero),
        'podas': len(podas),
        'valores_unicos': sorted(set([int(r['resultado']) for r in enteros_obtenidos])) if enteros_obtenidos else []
    }

# Realizar análisis de los resultados
estadisticas = analizar_resultados(resultados_obtenidos)


ANÁLISIS DE RESULTADOS
Total de juegos evaluados: 20
Resultados enteros obtenidos: 3
Divisiones por cero: 0
Evaluaciones con poda temprana: 0

VALORES ENTEROS ENCONTRADOS:
  Juego 2: -1.0
  Juego 3: 1.0
  Juego 11: 2.0

Valore únicos enteros: [-1, 1, 2]
Cantidad de valores únicos: 3


Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

**Referencias utilizadas:**

1. Documentación oficial de Python - https://docs.python.org/3/library/itertools para el uso de:
   - `itertools.permutations()` para generar permutaciones sin repetición
   - `math.perm()` y `math.factorial()` para cálculos combinatorios
   - Manejo de excepciones (try/except) para división por cero

2. Artículo sobre evaluación de expresiones matemáticas en Python: https://realpython.com/python-eval-function/

3. Temario de la asignatura y apuntes de clase:  
   - Algoritmos de fuerza bruta y búsqueda exhaustiva
   - Técnicas de poda temprana para optimización
   - Análisis de complejidad temporal y espacial



Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta

**Líneas futuras de investigación y variaciones del problema:**

Podemos pensar en extender el espacio de soluciones considerando más cifras (por ejemplo, incluir el 0 o usar todas las cifras del 0 al 9) y permitir expresiones más largas con más operadores. También sería interesante introducir nuevas operaciones como la potenciación, el módulo o incluso funciones matemáticas como raíces cuadradas, lo que abriría nuevas posibilidades de cálculo y aumentaría significativamente la riqueza del conjunto de resultados obtenibles.

Otra variación interesante es permitir el uso de paréntesis, lo cual cambia completamente la forma de evaluar las expresiones y añade una nueva capa de complejidad tanto en la representación de las soluciones como en los algoritmos utilizados. Asimismo, podríamos estudiar versiones del problema donde se permita repetir cifras u operadores, lo que modificaría drásticamente el espacio de búsqueda.

Desde el punto de vista de la aplicación práctica, también podemos reformular el problema como uno de búsqueda de un objetivo específico, en lugar de explorar todos los valores alcanzables. Esto nos llevaría a replantear el algoritmo desde una perspectiva de optimización: encontrar la expresión que se acerque más a un valor dado. Este cambio de enfoque abre la puerta al uso de técnicas como algoritmos genéticos, hill climbing o backtracking con heurísticas, especialmente útiles cuando el tamaño del problema crece.

Para mejorar el rendimiento del algoritmo, se pueden implementar estrategias como programación dinámica para evitar recalcular expresiones parciales, o poda temprana avanzada basada en modelos predictivos del comportamiento numérico.